In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

    Сайт коллекции музея:  https://metmuseum.org/

    API  часть сайта:   https://collectionapi.metmuseum.org/public/collection/

In [2]:
# обращаемся к сайту и записываем ответ
# r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=QUERY')
# res = r.json()

In [3]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/search?q='
IDs = 'sunflowers'     #  картины с подсолнухами
r = requests.get(url+IDs)
res = r.json()

In [4]:
# эксперимент: проверить наличие определенных данных
x = (437329, 20149, 436524, 2032)
for i in res['objectIDs']:
    if i in x: 
        print(i)

436524
2032
437329


In [5]:
res['total']

87

In [6]:
res.keys()

dict_keys(['total', 'objectIDs'])

In [7]:
len(res['objectIDs'])

87

In [8]:
#  ищем определеныый объект по его OBJECT_ID
#  r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/OBJECT_ID')

r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')
obj_test = r_test.json()

In [9]:
obj_test.keys()

dict_keys(['objectID', 'isHighlight', 'accessionNumber', 'accessionYear', 'isPublicDomain', 'primaryImage', 'primaryImageSmall', 'additionalImages', 'constituents', 'department', 'objectName', 'title', 'culture', 'period', 'dynasty', 'reign', 'portfolio', 'artistRole', 'artistPrefix', 'artistDisplayName', 'artistDisplayBio', 'artistSuffix', 'artistAlphaSort', 'artistNationality', 'artistBeginDate', 'artistEndDate', 'artistGender', 'artistWikidata_URL', 'artistULAN_URL', 'objectDate', 'objectBeginDate', 'objectEndDate', 'medium', 'dimensions', 'measurements', 'creditLine', 'geographyType', 'city', 'state', 'county', 'country', 'region', 'subregion', 'locale', 'locus', 'excavation', 'river', 'classification', 'rightsAndReproduction', 'linkResource', 'metadataDate', 'repository', 'objectURL', 'tags', 'objectWikidata_URL', 'isTimelineWork', 'GalleryNumber'])

In [10]:
obj_test['GalleryNumber']

'825'

In [11]:
obj_test['department']

'European Paintings'

In [12]:
obj_test['objectName']

'Painting'

In [13]:
obj_test['artistDisplayName']

'Vincent van Gogh'

In [14]:
#  полная структура записи объекта
obj_test

{'objectID': 437980,
 'isHighlight': False,
 'accessionNumber': '49.30',
 'accessionYear': '1949',
 'isPublicDomain': True,
 'primaryImage': 'https://images.metmuseum.org/CRDImages/ep/original/DP130999.jpg',
 'primaryImageSmall': 'https://images.metmuseum.org/CRDImages/ep/web-large/DP130999.jpg',
 'additionalImages': [],
 'constituents': [{'constituentID': 161947,
   'role': 'Artist',
   'name': 'Vincent van Gogh',
   'constituentULAN_URL': 'http://vocab.getty.edu/page/ulan/500115588',
   'constituentWikidata_URL': 'https://www.wikidata.org/wiki/Q5582',
   'gender': ''}],
 'department': 'European Paintings',
 'objectName': 'Painting',
 'title': 'Cypresses',
 'culture': '',
 'period': '',
 'dynasty': '',
 'reign': '',
 'portfolio': '',
 'artistRole': 'Artist',
 'artistPrefix': '',
 'artistDisplayName': 'Vincent van Gogh',
 'artistDisplayBio': 'Dutch, Zundert 1853–1890 Auvers-sur-Oise',
 'artistSuffix': '',
 'artistAlphaSort': 'Gogh, Vincent van',
 'artistNationality': 'Dutch',
 'artistB

Теперь напишем цикл, с помощью которого будет собрана информация об объектах, которые были получены на 2 шаге.(res['objectIDs']). Результаты запишем в датафрейм, а именно – следующие параметры:

     objectID – id объекта
    title – название
    artistDisplayName – автор
    department – департамент
    objectBeginDate – дата (начало)
    objectEndDate – дата (конец)
    period – название периода
    objectName – название/категория объекта
    culture – культура

In [15]:
# пример обхода списка, используем дальше
categ = ('objectID','title', 'artistDisplayName', 'department','objectBeginDate',\
         'objectEndDate','period','objectName','culture')

for j in categ:
        print(j)

objectID
title
artistDisplayName
department
objectBeginDate
objectEndDate
period
objectName
culture


In [16]:
#  1. вариант:  здесь получаем строки Index, но их неудобно потом объединять...
IDs = 437980
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

req = requests.get(url+str(IDs))
obj = req.json()
#print(url+'IDs',obj_cat)
result = [(obj[j]) for j in categ]    
df1 = pd.Index(result)

In [17]:
df1

Index([              437980,          'Cypresses',   'Vincent van Gogh',
       'European Paintings',                 1889,                 1889,
                         '',           'Painting',                   ''],
      dtype='object')

In [18]:
# 2 вариант:  здесь уже получаем строки DataFrame, используем этот вариант

IDs = 437980
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

dfnew = pd.DataFrame()
req = requests.get(url+str(IDs))
obj = req.json()
for j in categ:
    dfnew[j] = [obj[j]]

dfnew

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,437980,Cypresses,Vincent van Gogh,European Paintings,1889,1889,,Painting,


In [19]:
# объеденяем всё в один DataFrame, добавляем обработчик ошибок - каких то параметров может не быть в выдаче

categ = ('objectID','title', 'artistDisplayName', 'department',\
         'objectBeginDate','objectEndDate','period','objectName','culture')
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

df = pd.DataFrame()
for IDs in res['objectIDs']:
    req = requests.get(url+str(IDs))
    obj = req.json()
    #print(url+'IDs',obj_cat)
    try:
        for j in categ:
            dfnew[j] = [obj[j]]
            
    except:
            continue
    df = df.append(dfnew)

In [20]:
df

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,436524,Sunflowers,Vincent van Gogh,European Paintings,1887,1887,,Painting,
0,484935,Sunflowers,Edward McKnight Kauffer,Modern and Contemporary Art,1921,1921,,Painting,
0,437112,Bouquet of Sunflowers,Claude Monet,European Paintings,1881,1881,,Painting,
0,210191,Vase with sunflowers,Rozenburg Plateelfabriek,European Sculpture and Decorative Arts,1896,1896,,Vase,
0,431264,Sunflowers,Lily Converse,Drawings and Prints,1941,1941,,Print,
...,...,...,...,...,...,...,...,...,...
0,724844,Design for Embroidery with Two Large Garlands ...,Italian School,Drawings and Prints,1845,1905,,Drawing Ornament & Architecture,
0,437980,Cypresses,Vincent van Gogh,European Paintings,1889,1889,,Painting,
0,834585,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,
0,834580,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,


In [21]:
df.culture.unique()

array(['', 'American', 'Japan', 'British', 'Ancestral Pueblo'],
      dtype=object)

Поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat": запишите id объектов в cat_ids. Используйте цикл, чтобы получить данные о первой 1000 объектов из списка.

In [22]:
# !  search?departmentId=6&q=cat

r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()

In [23]:
slice_100 = cats['objectIDs'][0:100]

In [24]:
categ = ('objectID','title', 'artistDisplayName', 'department','objectBeginDate','objectEndDate','period','objectName','culture')
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

cats = pd.DataFrame()
for IDs in slice_100:
    req = requests.get(url+str(IDs))
    obj = req.json()
    #print(url+'IDs',obj_cat)
    try:
        for j in categ:
            dfnew[j] = [obj[j]]
            
    except:
            continue
    cats = cats.append(dfnew)

In [25]:
cats

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,49698,Cat,Zhang Yuguang,Asian Art,1900,1968,,Folding fan mounted as an album leaf,China
0,49470,Cat,Wang Li,Asian Art,1813,1879,Qing dynasty (1644–1911),Hanging scroll,China
0,53222,Musk Cat,Uto Gyoshi,Asian Art,1550,1599,Muromachi period (1392–1573),Hanging scroll,Japan
0,36221,Cat,Wang Yun,Asian Art,1900,1933,,Hanging scroll,China
0,60873,Burial Crown,,Asian Art,960,1279,Song dynasty (960–1279) or later,Burial crown,China
...,...,...,...,...,...,...,...,...,...
0,70582,Dye-Patterned Silk,,Asian Art,1600,1699,,Lining,India
0,73158,Head of Buddha,,Asian Art,600,699,pre-Angkor period,Head,Southern Cambodia
0,671014,Three Poems from the Later Collection of Japan...,Monk Saigyō,Asian Art,1167,1185,Heian period (794–1185),Page from booklet,Japan
0,670965,Writing Table (Bundai) with Melons and Squirrels,,Asian Art,1600,1649,Edo period (1615–1868),Writing table,Japan


In [26]:
cats.period.value_counts()

Edo period (1615–1868)                         30
                                               19
Qing dynasty (1644–1911)                       15
Meiji period (1868–1912)                        7
Edo (1615–1868) or Meiji period (1868–1912)     7
pre-Angkor period                               6
Han dynasty (206 B.C.–A.D. 220)                 3
Western Han dynasty (206 B.C.–A.D. 9)           1
Momoyama period (1573–1615)                     1
Satavahana-Ikshvaku                             1
Kushan period                                   1
Song dynasty (960–1279) or later                1
Ming dynasty (1368–1644)                        1
Ming dynasty (?) (1368–1644)                    1
Eastern Han dynasty (25–200)                    1
Central Javanese period                         1
Mon-Dvaravati period                            1
Khmer Empire (802–1431)                         1
Muromachi period (1392–1573)                    1
Heian period (794–1185)                         1
